In [42]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target

c:\Users\siddh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [62]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
num_particles = 16
num_iterations = 10
num_features = 28
col_len = 28
max_value = 783
std_dev = 150
mean = max_value / 2
pop_size = (num_particles, col_len)

population = np.random.normal(mean, std_dev, pop_size).astype(int)
population[population > max_value] = max_value
population[population < 0] = 0

In [64]:
len(population)

16

In [70]:
def pso_feature_selection(X_train, y_train, X_test, y_test, population, num_iterations):
    num_particles= population.shape[0]
    max_value = X_train.shape[1] - 1

    c1 = c2 = 1.49

    global_best = None
    global_best_accuracy = 0.0

    p_best_positions = population.copy()
    p_best_fitness_values = np.zeros(num_particles)

    velocity = np.random.randint(0, 28, size=(num_particles, num_features))

    for iteration in range(1, num_iterations + 1):
        print(global_best)
        print()
        print(global_best_accuracy)
        w_max = 0.41
        w_min = 0.4
        w = w_max - ((w_max - w_min) * iteration) / num_iterations

        for i in range(num_particles):
            print(global_best_accuracy)
            particle = population[i]

            X_train_ind = X_train.iloc[:,particle]
            rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)
            rf_classifier.fit(X_train_ind, y_train)

            y_pred = rf_classifier.predict(X_test.iloc[:,particle])

            accuracy = accuracy_score(y_test, y_pred)

            if accuracy > p_best_fitness_values[i]:
                p_best_positions[i] = particle.copy()
                p_best_fitness_values[i] = accuracy

            if accuracy > global_best_accuracy:
                global_best = particle.copy()
                global_best_accuracy = accuracy

        for i in range(num_particles):
            velocity_term = w * velocity[i]
            personal_best_term = c1 * np.random.rand() * (p_best_positions[i] - population[i])
            global_best_term = c2 * np.random.rand() * (global_best - population[i])

            velocity[i] = velocity_term + personal_best_term + global_best_term
            population[i] = np.round(population[i] + velocity[i]).astype(int)

            population[i][population[i] > max_value] = max_value
            population[i][population[i] < 0] = 0

    return global_best

In [71]:
result = pso_feature_selection(X_train, y_train, X_test, y_test, population, num_iterations)

None

0.0
0.0
0.6413571428571428
0.7983571428571429
0.7983571428571429
0.7983571428571429
0.7983571428571429
0.7983571428571429
0.7983571428571429
0.7983571428571429
0.7983571428571429
0.7983571428571429
0.7983571428571429
0.7983571428571429
0.8077142857142857
0.8077142857142857
0.8077142857142857
[264   4 355 234 503 430 449 608 214 283 201 783 378 349 439 322 382 540
 599 384 365 223 234 517 428 296 628 147]

0.8077142857142857
0.8077142857142857
0.8077142857142857
0.8077142857142857
0.8077142857142857
0.8077142857142857
0.8077142857142857
0.8077142857142857
0.8077142857142857
0.8077142857142857
0.8077142857142857
0.8077142857142857
0.8077142857142857
0.8077142857142857
0.8116428571428571
0.8116428571428571
0.8116428571428571
[273  12 361 240 512 435 455 610 220 292 210 783 379 349 443 330 391 544
 599 385 367 227 234 523 436 303 632 152]

0.8116428571428571
0.8116428571428571
0.8116428571428571
0.8116428571428571
0.8116428571428571
0.8116428571428571
0.8116428571428571
0.81164285714

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train.iloc[:,result], y_train)

y_pred = rf_classifier.predict(X_test.iloc[:,result])
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8570952380952381


In [73]:
new_population = np.random.randint(0, 783, size=28)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train.iloc[:,new_population], y_train)

y_pred = rf_classifier.predict(X_test.iloc[:,new_population])
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6699047619047619
